In [ ]:
import os
import sys
import logging
from tqdm import tqdm
import argparse

import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

import torch
import torch.nn.functional as F
import torch.optim as optim

from torch_geometric.graphgym import train
from torch_geometric.loader import DataLoader
from torch_geometric.utils import scatter
from torch_geometric.nn.pool import global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data, Batch

from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from ogb.graphproppred import PygGraphPropPredDataset

/home/vedant/miniconda3/envs/hbt/lib/python3.11/site-packages/torch_geometric/graphgym/config.py:19: UserWarning: Could not define global config object. Please install 'yacs' via 'pip install yacs' in order to use GraphGym
  warnings.warn("Could not define global config object. Please install "
/home/vedant/miniconda3/envs/hbt/lib/python3.11/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' via  'pip install pytorch_lightning' in order to use GraphGym
  warnings.warn("Please install 'pytorch_lightning' via  "


In [ ]:
sys.path.append(os.path.dirname(rf"/home/vedant/hyperbolic-transformer/Hypformer/hypformer.py"))
from hypformer import HypFormer

In [ ]:
def prepare_dataloaders(batch_size):
    dataset = PygGraphPropPredDataset(name = 'ogbg-molhiv')
    splits = dataset.get_idx_split()
    train_ds = dataset[splits['train']]
    valid_ds = dataset[splits['valid']]
    test_ds  = dataset[splits['test']]

    train_loader = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
    valid_loader = DataLoader(valid_ds, batch_size = batch_size, shuffle = False)
    test_loader = DataLoader(test_ds,  batch_size = batch_size, shuffle = False)
    return train_loader, valid_loader, test_loader, dataset.num_features

In [ ]:
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    i = 0

    for data in tqdm(loader, desc="Training"):
        # aggregate 7‑dim edge_attr to node features
        src, _ = data.edge_index
        data.x = scatter(data.edge_attr, src,
                            dim = 0,
                            dim_size = data.num_nodes,
                            reduce = 'sum')
        data = data.to(device)

        optimizer.zero_grad()
        # HypFormer returns [num_nodes, out_channels]
        node_logits = model(data.x)
        # pool to graph‑level logits [batch_size, out_channels]
        graph_logits = global_mean_pool(node_logits, data.batch)
        loss = F.cross_entropy(graph_logits, data.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # del data, loss, node_logits, graph_logits
        # torch.cuda.empty_cache()
        # print(i)
        
        i += 1
    return total_loss / len(loader)

In [ ]:
@torch.no_grad()
def get_eval_loss(model, loader, device):
    model.eval()
    total_loss = 0.0
    i = 0

    for data in loader:
        # aggregate 7‑dim edge_attr to node features
        src, _ = data.edge_index
        data.x = scatter(data.edge_attr, src,
                            dim = 0,
                            dim_size = data.num_nodes,
                            reduce = 'sum')
        data = data.to(device)

        # HypFormer returns [num_nodes, out_channels]
        node_logits = model(data.x)
        # pool to graph‑level logits [batch_size, out_channels]
        graph_logits = global_mean_pool(node_logits, data.batch)
        loss = F.cross_entropy(graph_logits, data.y.view(-1))
        total_loss += loss.item()
        i += 1

    return total_loss / len(loader)

In [ ]:
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    y_true = []
    y_prob = []
    print(device)

    for data in loader:
        src, _ = data.edge_index
        data.x = scatter(data.edge_attr, src,
                              dim = 0,
                              dim_size = data.num_nodes,
                              reduce = 'sum')
        data = data.to(device)

        # forward pass
        node_logits  = model(data.x)
        graph_logits = global_mean_pool(node_logits, data.batch)

        if graph_logits.shape[1] == 1:
            probs = torch.sigmoid(graph_logits.view(-1))
        else:
            probs = torch.softmax(graph_logits, dim=1)[:, 1]

        y_true.append(data.y.view(-1).cpu().numpy())
        y_prob.append(probs.cpu().numpy())

    # concatenate all batches
    y_true = np.concatenate(y_true, axis = 0)
    y_prob = np.concatenate(y_prob, axis = 0)
    
    return roc_auc_score(y_true, y_prob)

In [6]:
# get_eval_loss(model, train_loader, device)

In [ ]:
parser = argparse.ArgumentParser()

# Original args
parser.add_argument('--batch_size', type = int, default = 16) # TODO: Try larger batch size
parser.add_argument('--hidden_dim', type = int, default = 128)
parser.add_argument('--trans_num_layers', type = int, default = 2)
parser.add_argument('--trans_num_heads', type = int, default = 4)
parser.add_argument('--trans_dropout', type = float, default = 0.2)
parser.add_argument('--lr', type = float, default = 1e-5)
parser.add_argument('--weight_decay', type = float, default = 1e-4)
parser.add_argument('--epochs', type = int, default = 5)
parser.add_argument('--k_in', type = float, default = 1.0)
parser.add_argument('--k_out', type = float, default = 1.0)
parser.add_argument('--decoder_type', type = str, default = 'hyp')
parser.add_argument('--device', type = str, default = 'cuda:0')
parser.add_argument('--add_positional_encoding', action = 'store_true')
parser.add_argument('--attention_type', type = str, default = 'full')
parser.add_argument('--power_k', type = int, default = 2)
parser.add_argument('--trans_heads_concat', action = 'store_true')
parser.add_argument('--aggregate', action = 'store_true')
parser.add_argument('--extra_hyp_linears', type = int, default = 10)
args = parser.parse_args([])

args_dict = vars(args)
print(args_dict)

{'batch_size': 16, 'hidden_dim': 128, 'trans_num_layers': 2, 'trans_num_heads': 4, 'trans_dropout': 0.2, 'lr': 1e-05, 'weight_decay': 0.0001, 'epochs': 5, 'k_in': 1.0, 'k_out': 1.0, 'decoder_type': 'hyp', 'device': 'cuda:0', 'add_positional_encoding': False, 'attention_type': 'full', 'power_k': 2, 'trans_heads_concat': False, 'aggregate': False, 'extra_hyp_linears': 10}


In [8]:
train_loader, valid_loader, test_loader, in_channels = prepare_dataloaders(args.batch_size)

In [ ]:
# torch.cuda.empty_cache()

In [ ]:

model = HypFormer(
    in_channels = 3,
    hidden_channels = 1024,
    extra_dims = [64, 256, 512, 1024, 2048, 4096, 2048, 1024, 512, 1024, 1024],
    out_channels = 2,  # binary classification
    trans_num_layers = 2,
    trans_num_heads = 8,
    trans_dropout = 0.1,
    trans_use_bn = True,
    trans_use_residual = True,
    trans_use_weight = True,
    trans_use_act = True,
    args = args
)

In [13]:
device = "cuda"
model = model.to(device)

In [14]:
sum(p.numel() for p in model.parameters() if p.requires_grad)/10**6


132.052048

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

optimizer = optim.AdamW(model.parameters(), lr = args.lr, weight_decay = args.weight_decay)
scheduler = CosineAnnealingLR(optimizer, T_max = args.epochs)

In [ ]:
# torch.cuda.empty_cache()  # Releases all unused cached memory

In [ ]:
val_roc = []
val_loss = []
train_loss = []
# val_acc = []
best_val = 0

for epoch in range(1, 100):
    print("Epoch started!!")
    loss = train_epoch(model, train_loader, optimizer, device)
    print("Epoch done!!")
    val_rocauc = evaluate(model, valid_loader, device)
    print("Eval ROC done!!")
    current_lr = scheduler.get_last_lr()[0]
    print("Gradient done!!")
    val_loss_ = get_eval_loss(model, valid_loader, device)
    print("Val loss done!!")
    val_loss.append(val_loss_)
    # val_acc.append(val_acc_)

    log_msg = (
        f"[Epoch {epoch:03d}] "
        f"LR: {current_lr:.6f} "
        f"Train Loss: {loss:.4f} "
        f"Val ROC-AUC: {val_rocauc:.4f}"
        f"Val Loss:{val_loss_:.4f}"
        # f"Val accuracy: {val_acc_:.4f}"
    )
    print(log_msg)

    # Save best model
    if val_rocauc > best_val:
        best_val = val_rocauc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_rocauc': val_rocauc
        }, "checkpoints/best_model.pt")
        
        print(f"New best model saved at epoch {epoch} with Val ROC-AUC: {val_rocauc:.4f}")

    val_roc.append(val_rocauc)
    train_loss.append(loss)

    # Step the scheduler
    scheduler.step()

final_msg = f"Training finished. Best Validation ROC-AUC: {best_val:.4f}"
print(final_msg)
logging.info(final_msg)

Epoch started!!


Training: 100%|██████████| 2057/2057 [10:59<00:00,  3.12it/s]


Epoch done!!
cuda
Eval ROC done!!
Gradient done!!
Val loss done!!
[Epoch 001] LR: 0.000010 Train Loss: 0.1724 Val ROC-AUC: 0.4980Val Loss:0.0975
New best model saved at epoch 1 with Val ROC-AUC: 0.4980
Epoch started!!


Training: 100%|██████████| 2057/2057 [11:02<00:00,  3.11it/s]


Epoch done!!
cuda
Eval ROC done!!
Gradient done!!
Val loss done!!
[Epoch 002] LR: 0.000009 Train Loss: 0.1601 Val ROC-AUC: 0.5084Val Loss:0.0985
New best model saved at epoch 2 with Val ROC-AUC: 0.5084
Epoch started!!


...

Training: 100%|██████████| 2057/2057 [06:05<00:00,  5.62it/s]


Epoch done!!
cuda
Eval ROC done!!
Gradient done!!
Val loss done!!
[Epoch 093] LR: 0.000007 Train Loss: 0.1590 Val ROC-AUC: 0.4815Val Loss:0.1015
Epoch started!!


Training: 100%|██████████| 2057/2057 [06:11<00:00,  5.54it/s]


Epoch done!!
cuda
Eval ROC done!!
Gradient done!!
Val loss done!!
[Epoch 094] LR: 0.000003 Train Loss: 0.1589 Val ROC-AUC: 0.4816Val Loss:0.1018
Epoch started!!


Training:  34%|███▎      | 693/2057 [02:04<04:42,  4.84it/s]

In [ ]:
os.path.getsize("checkpoints/best_model.pt") / (1024**2)

1471.1783208847046